In [4]:
import pandas as pd
import random
import numpy as np

In [5]:
#Hasta 15 de Abril (para entrenar)

postulantes_edu_df = pd.read_csv('fiuba_1_postulantes_educacion_h15.csv')
postulantes_gye_df = pd.read_csv('fiuba_2_postulantes_genero_y_edad_h15.csv')
vistas_df = pd.read_csv('fiuba_3_vistas_h15.csv')
postulaciones_df = pd.read_csv('fiuba_4_postulaciones_h15.csv')
avisos_online_df = pd.read_csv('fiuba_5_avisos_online_h15.csv')
avisos_detalle_df = pd.read_csv('fiuba_6_avisos_detalle_h15.csv')

In [6]:
#Originales TP1 (para entrenar)

postulantes_edu_df2 = pd.read_csv('fiuba_1_postulantes_educacion.csv')
postulantes_gye_df2 = pd.read_csv('fiuba_2_postulantes_genero_y_edad.csv')
vistas_df2 = pd.read_csv('fiuba_3_vistas.csv')
postulaciones_df2 = pd.read_csv('fiuba_4_postulaciones.csv')
avisos_online_df2 = pd.read_csv('fiuba_5_avisos_online.csv')
avisos_detalle_df2 = pd.read_csv('fiuba_6_avisos_detalle.csv')

In [7]:
#Realizo el Merge del tipo outer de los datos
#Originales TP1 y Datos hasta 15 de abril

postulantes_edu_df = postulantes_edu_df.merge(postulantes_edu_df2,how='outer')
postulantes_gye_df = postulantes_gye_df.merge(postulantes_gye_df2,how='outer')
vistas_df = vistas_df.merge(vistas_df2,how='outer')
postulaciones_df = postulaciones_df.merge(postulaciones_df2,how='outer')
avisos_online_df = avisos_online_df.merge(avisos_online_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_df2,how='outer')

In [8]:
#Desde 15 de abril (para entrenar)

postulantes_edu_df2 = pd.read_csv('fiuba_1_postulantes_educacion_d15.csv')
postulantes_gye_df2 = pd.read_csv('fiuba_2_postulantes_genero_y_edad_d15.csv')
vistas_df2 = pd.read_csv('fiuba_3_vistas_d15.csv')
avisos_detalle_df2 = pd.read_csv('fiuba_6_avisos_detalle_d15.csv')

In [9]:
#Avisos detalle que faltaban
avisos_detalle_missing = pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

In [10]:
#Mergeamos todo

postulantes_edu_df = postulantes_edu_df.merge(postulantes_edu_df2,how='outer')
postulantes_gye_df = postulantes_gye_df.merge(postulantes_gye_df2,how='outer')
vistas_df = vistas_df.merge(vistas_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_missing,how='outer')

# Libero Espacio de la Memoria

In [11]:
del postulantes_edu_df2
del postulantes_gye_df2
del vistas_df2
del avisos_detalle_df2

# Trabajamos con la educacion

Asignamos un valor para cada estado de la educacion, Graduado->2, En Curso->1, Abandonado->0

In [20]:
def setCategory(c):
    if c == 'Graduado':
        return 2
    elif c == 'En Curso':
        return 1
    return 0 

In [21]:
postulantes_edu_df['valor_estado'] = postulantes_edu_df['estado'].apply(setCategory)

In [22]:
postulantes_edu_df.head(2)

,idpostulante,nombre,estado,valor_estado
0,ZjlZ,Master,En Curso,1
1,NdJl,Posgrado,En Curso,1
2,5kNq,Otro,En Curso,1
3,8rYD,Master,En Curso,1
4,1Wvj,Universitario,En Curso,1


Al igual que hicimos con los estados de la educacion, lo hacemos con el nivel de educacion.

In [23]:
postulantes_edu_df['nombre'].value_counts()

Universitario        250850
Secundario           244689
Terciario/Técnico    113421
Otro                  53363
Posgrado              20624
Master                10074
Doctorado               620
Name: nombre, dtype: int64

In [24]:
def setCategory(c):
    
    if c == 'Doctorado':
        return 12
    elif c == 'Master':
        return 10
    elif c == 'Posgrado':
        return 8
    elif c == 'Universitario':
        return 6
    elif c == 'Terciario/Técnico':
        return 4
    elif c == 'Secundario':
        return 2
    return 0 

In [25]:
postulantes_edu_df['valor_nombre'] = postulantes_edu_df['nombre'].apply(setCategory)

In [26]:
postulantes_edu_df.head(2)

,idpostulante,nombre,estado,valor_estado,valor_nombre
0,ZjlZ,Master,En Curso,1,10
1,NdJl,Posgrado,En Curso,1,8


Ahora unificamos los datos obtenidos, creando una escala donde a mayor educacion, mayor peso. Considerando el tipo de educacion y su estado.
Este nuevo feature llamado valoracion_edu, reemplazará a 'nombre'y 'estado'

In [27]:
postulantes_edu_df['valoracion_edu'] = postulantes_edu_df['valor_estado'] + postulantes_edu_df['valor_nombre']
postulantes_edu_df.drop(columns=['valor_estado','valor_nombre','nombre','estado'],axis=1,inplace=True)
postulantes_edu_df.sort_values(by='valoracion_edu',ascending=False,inplace=True)
postulantes_edu_df.drop_duplicates(inplace=True)
postulantes_edu_df.head()

,idpostulante,valoracion_edu
318724,N9AN45,14
128733,vV9Ewjj,14
337840,aMbWjD,14
398665,OqmK81N,14
14753,JBrVvBp,14


Para la informacion sobre el genero, unificamos los valores 0.0 con los NO_DECLARA

In [28]:
postulantes_gye_df.sexo.value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
0.0                7
Name: sexo, dtype: int64

In [29]:
def setCategory(c):
    if c == '0.0':
        return 'NO_DECLARA'
    return c

In [30]:
postulantes_gye_df['sexo'] = postulantes_gye_df['sexo'].apply(setCategory)

In [31]:
postulantes_gye_df.sexo.value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25943
Name: sexo, dtype: int64

# Unificamos los dataframes GYE con EDU

In [32]:
postulantes_filtrados_df = postulantes_gye_df.merge(postulantes_edu_df,on='idpostulante',how='outer')

print('Shape',postulantes_filtrados_df.shape)

print('Unique Shape',postulantes_filtrados_df['idpostulante'].unique().shape)

(738053, 4)


(504407,)

In [33]:
postulantes_filtrados_df.head(2)

,idpostulante,fechanacimiento,sexo,valoracion_edu
0,6MM,1985-01-01,MASC,NaN
1,Nzz,NaN,NO_DECLARA,NaN


Asumimos que los usuarios que no completaron su valor, tienen educacion minima, es decir, secundario abandonado.
Las fechas de nacimiento nulas se completa con la edad promedio de los usuarios registrados(31 años).

In [34]:
postulantes_filtrados_df['sexo'].fillna('NO_DECLARA',inplace=True)
postulantes_filtrados_df['valoracion_edu'].fillna(2, inplace=True)
postulantes_filtrados_df['fechanacimiento'].fillna('1987-01-01', inplace=True)

In [35]:
postulantes_filtrados_df.head(2)

,idpostulante,fechanacimiento,sexo,valoracion_edu
0,6MM,1985-01-01,MASC,2.0
1,Nzz,1987-01-01,NO_DECLARA,2.0
2,ZX1,1987-01-01,NO_DECLARA,2.0
3,Nq5,1987-01-01,NO_DECLARA,2.0
4,ebE,1952-07-07,MASC,10.0


Al ordenar alfabeticamente sexo, los No_declara quedan ultimos, por lo tanto, al hacer un drop_duplicates, nos quedamos con la primera aparicion de ese usuario.
De esta forma si un usuario se registro dos veces, queda el registro correcto sobre su genero.

In [37]:
postulantes_filtrados_df.sort_values(by='sexo',inplace=True)

In [38]:
postulantes_filtrados_df = postulantes_filtrados_df.drop_duplicates('idpostulante')
postulantes_filtrados_df.shape

(504407, 4)

In [39]:
postulantes_filtrados_df.sexo.value_counts()

FEM           251226
MASC          227832
NO_DECLARA     25349
Name: sexo, dtype: int64

In [40]:
postulantes_filtrados_df.idpostulante.unique().shape

(504407,)

Las edades incorrectas son reemplazadas por al edad promedio.

In [43]:
postulantes_edad_trucha = postulantes_filtrados_df[ (postulantes_filtrados_df['fechanacimiento'].str.startswith('0')) == True]
postulantes_edad_trucha['fechanacimiento'] = '1987-01-01'
postulantes_edad_trucha.head()


/home/luciano/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,idpostulante,fechanacimiento,sexo,valoracion_edu
299748,8M2R6pz,1987-01-01,FEM,2.0
311804,0zkAXBE,1987-01-01,FEM,7.0
341475,VNrP8EE,1987-01-01,FEM,2.0
347580,xkdB9JK,1987-01-01,FEM,8.0
324043,VNxMRxj,1987-01-01,FEM,2.0


Filtro la edad incorrecta y hago un append de la edad corregida.

In [44]:
postulantes_filtrados_df = postulantes_filtrados_df[ (postulantes_filtrados_df['fechanacimiento'].str.startswith('0')) == False]
postulantes_filtrados_df = postulantes_filtrados_df.append(postulantes_edad_trucha)
postulantes_filtrados_df.head(2)


,idpostulante,fechanacimiento,sexo,valoracion_edu
369026,5mPYWjp,1989-05-01,FEM,4.0
588433,xkdxkDz,1993-08-28,FEM,5.0
303865,EzE23E4,1999-04-23,FEM,4.0
303863,6r5OwJO,1997-06-07,FEM,4.0
588434,lDL3D1X,1997-07-17,FEM,2.0


Verifico que no se hayan perdido datos.

In [45]:
postulantes_filtrados_df.shape

(504407, 4)

Calculo la edad y reemplazo la columna 'fechanacimiento' por 'edad'

In [46]:
postulantes_filtrados_df.loc[:,'fechanacimiento'] = pd.to_datetime(postulantes_filtrados_df['fechanacimiento']) 
postulantes_filtrados_df.loc[:,'edad'] = (2018 - postulantes_filtrados_df.fechanacimiento.dt.year)
postulantes_filtrados_df.drop(columns='fechanacimiento',axis=1,inplace=True)
postulantes_filtrados_df.head(2)

,idpostulante,sexo,valoracion_edu,edad
369026,5mPYWjp,FEM,4.0,29
588433,xkdxkDz,FEM,5.0,25
303865,EzE23E4,FEM,4.0,19
303863,6r5OwJO,FEM,4.0,21
588434,lDL3D1X,FEM,2.0,21


# Avisos Detalle

In [48]:
avisos_detalle_df.head(2)

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,1,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,1,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan


In [49]:
print(avisos_detalle_df.shape)
print(avisos_detalle_df['idaviso'].unique().shape)

(25901, 11)
(25288,)


Sacamos los repetidos de avisos_detalle

In [50]:
avisos_detalle_filtrados_df = avisos_detalle_df.drop_duplicates('idaviso')

(25901, 11)

In [51]:
del avisos_detalle_df

Hago un drop de las columnas que no voy a utilizar.

In [53]:
avisos_detalle_filtrados_df = avisos_detalle_filtrados_df.drop(columns=['idpais','mapacalle','ciudad','descripcion'],axis=1)

In [54]:
avisos_detalle_filtrados_df.head(2)

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,Software Engineer,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Business Developer Manager,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


Unificamos zonas cercanas

In [55]:
avisos_detalle_filtrados_df.nombre_zona.value_counts()

Gran Buenos Aires              23018
Capital Federal                 2182
Buenos Aires (fuera de GBA)       57
GBA Oeste                         10
La Plata                           4
Mendoza                            3
Ciudad de Mendoza                  3
Rosario                            2
Cordoba                            2
Neuquen                            1
Corrientes                         1
Santa Fe                           1
Catamarca                          1
Santa Cruz                         1
San Juan                           1
Tucuman                            1
Name: nombre_zona, dtype: int64

In [56]:
def agrupacionZona(c):
    
    if c == 'Capital Federal' or c=='GBA Oeste':
        return 'Gran Buenos Aires'
    
    elif c == 'Ciudad de Mendoza':
        return 'Mendoza'
    
    return c 

In [57]:
avisos_detalle_filtrados_df['nombre_zona'] = avisos_detalle_filtrados_df['nombre_zona'].apply(agrupacionZona)

In [58]:
avisos_detalle_filtrados_df.nombre_zona.value_counts()

Gran Buenos Aires              25210
Buenos Aires (fuera de GBA)       57
Mendoza                            6
La Plata                           4
Rosario                            2
Cordoba                            2
Neuquen                            1
Corrientes                         1
Santa Fe                           1
Catamarca                          1
Santa Cruz                         1
San Juan                           1
Tucuman                            1
Name: nombre_zona, dtype: int64

# Postulaciones

Nos quedamos con los usuarios sobre los cuales tenemos informacion y ademas se postularon a un aviso.

In [60]:
postulaciones_df.head(2)

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


Comparamos los "usuarios" con los  postulantes.

In [61]:
print(postulantes_filtrados_df.idpostulante.unique().shape)
print(postulaciones_df.idpostulante.unique().shape)

(504407,)
(348883,)


Como resultado del siguiente merge tenemos los postulantes que coinciden en postulaciones y postulantes.
Observamos que hay usuarios que no se postularon a ningun aviso.

In [62]:
postulantes_filtrados_efectivos_df = postulaciones_df.merge(postulantes_filtrados_df,on='idpostulante',how='inner')
postulantes_filtrados_efectivos_df.idpostulante.unique().shape

(348883,)

In [64]:
postulantes_filtrados_efectivos_df =  postulantes_filtrados_efectivos_df.drop(['idaviso','fechapostulacion'],axis='columns')
postulantes_filtrados_efectivos_df = postulantes_filtrados_efectivos_df.reset_index()
postulantes_filtrados_efectivos_df =  postulantes_filtrados_efectivos_df.drop('index',axis='columns')
postulantes_filtrados_efectivos_df.head(2)

,idpostulante,sexo,valoracion_edu,edad
0,NjlD,FEM,6.0,37
1,ZaO5,FEM,6.0,40
2,NdJl,MASC,8.0,49
3,5kNq,MASC,1.0,45
4,Ez8J,MASC,6.0,50


In [66]:
postulantes_filtrados_efectivos_df.shape

(348883, 4)

# (NO CORRER) Armado del set de Postulaciones

Me quedo con los idavisos sobre los cuales tengo informacion

In [64]:
avisos = avisos_detalle_filtrados_df['idaviso'].to_frame()

Me quedo con los idpostulantes sobre los cuales tengo informacion

In [65]:
postulantes = postulantes_filtrados_df['idpostulante'].to_frame()

Hago una interseccion para quedarme con usuarios y avisos sobre los cuales tengo informacion de su postulacion

In [66]:
postulantes_df =  postulaciones_df.merge(avisos,how='inner',on='idaviso')
postulantes_df = postulantes_df.merge(postulantes,how='inner',on='idpostulante')

Armo el formato necesario para que me quede la dupla idpostulante-idaviso

In [67]:
postulantes_df =postulantes_df.groupby('idpostulante').agg({'idaviso':'value_counts'})
postulantes_df.columns=['postulacion']
postulantes_df['postulacion'] = 1 

Borramos para liberar memoria informacion que ya no utilizamos

In [ ]:
del avisos
del postulantes

In [69]:
postulantes_df.reset_index(inplace=True)

In [70]:
postulantes_df.head(2)

In [71]:
postulantes_df.idaviso.unique().shape

Ahora vamos a limitar la cantidad de postulaciones por usuario, para evitar aprender demasiado sobre pocos usuarios con muchas postulaciones. Elegimos limitarlas al promedio de postulaciones por usuario (20).

Agrupamos por idpostulante para poder acceder a cada usuario en particular de forma directa

In [72]:
grupo_postulantes = postulantes_df.groupby('idpostulante')

Ejemplo de cómo accedemos al usuario y aprovechamos el resultado para generar el DataFrame base.

In [75]:
postulantes_unicos = postulantes_df.idpostulante.unique()

In [73]:
postulantes_df = grupo_postulantes.get_group('0z5Dmrd').head(20).drop([0,1],axis=0)

In [76]:
for postulante in postulantes_unicos:
     postulantes_df = postulantes_df.append(grupo_postulantes.get_group(postulante).head(20))      

In [77]:
postulantes_df.head()

In [ ]:
postulantes_df.to_csv('postulaciones_limitadas.csv')

In [64]:
#postulantes_df = pd.read_csv('postulaciones_limitadas.csv')
#postulantes_df = postulantes_df.drop('Unnamed: 0',1)

In [65]:
postulantes_df.head(2)

,Unnamed: 0,idpostulante,idaviso,postulacion
0,0,0z5Dmrd,1112384041,1
1,1,0z5Dmrd,1112420060,1
2,2,0z5JW1r,1112257171,1
3,3,0z5JW1r,1112281879,1
4,4,0z5JW1r,1112292169,1


# Armado del set de No Postulaciones

Armarmos un set de no postulaciones asignando de forma random avisos a usuarios.
Mas adelante se verifica que esta no postulacion al azar no sea una postulacion efectiva o que el aviso no haya sido visto por el usuario.

In [83]:
vistas_df.head(2)

,idAviso,timestamp,idpostulante
0,1112400775,2018-04-06T01:43:47.445-0400,kPjr2K0
1,1112400775,2018-04-06T01:43:48.330-0400,kPjr2K0
2,1112426664,2018-04-06T01:43:50.042-0400,GNOwZmb
3,1112426664,2018-04-06T01:43:51.171-0400,GNOwZmb
4,1112445738,2018-04-06T01:43:46.371-0400,lDLxajz


In [84]:
vistas_df = vistas_df.drop('timestamp',axis=1)
vistas_df['lo_vio']=1
vistas_df.head(2)

,idAviso,idpostulante,lo_vio
0,1112400775,kPjr2K0,1
1,1112400775,kPjr2K0,1
2,1112426664,GNOwZmb,1
3,1112426664,GNOwZmb,1
4,1112445738,lDLxajz,1


In [85]:
vistas_df.columns = ['idaviso','idpostulante','lo_vio']


In [86]:
vistas_df = vistas_df.drop_duplicates(['idaviso','idpostulante'])

In [87]:
vistas_df.shape

(8597915, 3)

Asumimos que si un usuario no aparece en el DataFrame de vistas o en postulaciones, es un usuario inactivo. Por eso tomamos los usuarios que aparecen en alguno de los DataFrames mencionados

In [88]:
lista = []
usuarios_activos = vistas_df.drop_duplicates('idpostulante').merge(postulaciones_df.drop_duplicates('idpostulante').drop(['idaviso','fechapostulacion'],1),how='outer',on='idpostulante')
user_list = postulantes_filtrados_df.merge(usuarios_activos,how='inner',on='idpostulante').idpostulante.unique()
aviso_list = avisos_detalle_filtrados_df['idaviso'].unique()
index = 1
for n in range(3000000):
    user_index = random.randint(0,476360)
    aviso_index = random.randint(0,25287)
    lista.append((index,user_list[user_index],aviso_list[aviso_index],0))
    index+=1        

In [89]:
no_postulantes_df = pd.DataFrame(lista)

In [90]:
del lista

In [92]:
no_postulantes_df.head(2)

,0,1,2,3
0,1,NzrLwKJ,1112513555,0
1,2,N6qMwV,1111882745,0
2,3,Nzr3zvL,1112362954,0
3,4,jkj13j0,1112146217,0
4,5,ZDrmWxY,1112400202,0


In [93]:
no_postulantes_df.drop(0,axis='columns',inplace=True)

In [95]:
no_postulantes_df.columns= ['idpostulante','idaviso','postulacion']

In [94]:
no_postulantes_df.head()

,1,2,3
0,NzrLwKJ,1112513555,0
1,N6qMwV,1111882745,0
2,Nzr3zvL,1112362954,0
3,jkj13j0,1112146217,0
4,ZDrmWxY,1112400202,0


In [96]:
no_postulantes_df.shape

(3000000, 3)

# Armado de Set de Entrenamiento

Quitamos las no postulaciones que figuran como vistas.

In [99]:
entrenamiento_df = vistas_df.merge(no_postulantes_df,how='outer',on=['idpostulante','idaviso'])

In [100]:
entrenamiento_df.head()

,idaviso,idpostulante,lo_vio,postulacion
0,1112400775,kPjr2K0,1.0,NaN
1,1112426664,GNOwZmb,1.0,NaN
2,1112445738,lDLxajz,1.0,NaN
3,1112416123,owzZOqe,1.0,NaN
4,1112443938,NGk68l,1.0,NaN


In [101]:
entrenamiento_df['lo_vio'].hasnans

True

In [102]:
entrenamiento_df['postulacion'].fillna(0,inplace=True)
entrenamiento_df['lo_vio'].fillna(0,inplace=True)

In [103]:
entrenamiento_df['postulacion'] = entrenamiento_df['lo_vio'] + entrenamiento_df['postulacion']

In [104]:
entrenamiento_df.head()

,idaviso,idpostulante,lo_vio,postulacion
0,1112400775,kPjr2K0,1.0,1.0
1,1112426664,GNOwZmb,1.0,1.0
2,1112445738,lDLxajz,1.0,1.0
3,1112416123,owzZOqe,1.0,1.0
4,1112443938,NGk68l,1.0,1.0


In [105]:
entrenamiento_df = entrenamiento_df[entrenamiento_df['postulacion']==0]

In [106]:
entrenamiento_df = entrenamiento_df.drop('lo_vio',axis=1)

Juntamos los no_postulantes con los postulantes para armar el set de entrenamiento

In [107]:
entrenamiento_df = entrenamiento_df.merge(postulantes_df,how='outer',on=['idpostulante','idaviso'])

In [108]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion_x,postulacion_y
0,1112513555,NzrLwKJ,0.0,NaN
1,1111882745,N6qMwV,0.0,NaN
2,1112362954,Nzr3zvL,0.0,NaN
3,1112146217,jkj13j0,0.0,NaN
4,1112400202,ZDrmWxY,0.0,NaN


In [109]:
entrenamiento_df['postulacion_x'].fillna(0,inplace=True)
entrenamiento_df['postulacion_y'].fillna(0,inplace=True)

La idea es sumar las columnas duplicadas, ya que si habia una postulacion para esa misma dupla idaviso-idpostulante, predomina la postulacion(1) sobre la no_postulacion(0)

In [110]:
entrenamiento_df['postulacion'] = entrenamiento_df['postulacion_x'] + entrenamiento_df['postulacion_y']

In [111]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion_x,postulacion_y,postulacion
0,1112513555,NzrLwKJ,0.0,0.0,0.0
1,1111882745,N6qMwV,0.0,0.0,0.0
2,1112362954,Nzr3zvL,0.0,0.0,0.0
3,1112146217,jkj13j0,0.0,0.0,0.0
4,1112400202,ZDrmWxY,0.0,0.0,0.0


In [112]:
entrenamiento_df.drop(['postulacion_x','postulacion_y'],axis='columns',inplace=True)

In [113]:
entrenamiento_df['postulacion'].value_counts()

1.0    3277072
0.0    2998008
Name: postulacion, dtype: int64

In [114]:
entrenamiento_df['postulacion'] = pd.Categorical(entrenamiento_df['postulacion'])
entrenamiento_df['postulacion'] = entrenamiento_df['postulacion'].cat.codes
entrenamiento_df['postulacion'].value_counts()

1    3277072
0    2998008
Name: postulacion, dtype: int64

In [115]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion
0,1112513555,NzrLwKJ,0
1,1111882745,N6qMwV,0
2,1112362954,Nzr3zvL,0
3,1112146217,jkj13j0,0
4,1112400202,ZDrmWxY,0


In [116]:
entrenamiento_df.tail()

,idaviso,idpostulante,postulacion
6275075,1112274234,zvxzrWm,1
6275076,1112279048,zvxzrWm,1
6275077,1112280433,zvxzrWm,1
6275078,1112289195,zvxzrWm,1
6275079,1112290025,zvxzrWm,1


Sacamos pares idaviso-idpostulante duplicados y que aparecen en test_final

In [118]:
entrenamiento_df = entrenamiento_df.drop_duplicates(['idaviso','idpostulante'])

In [119]:
test_final_df = pd.read_csv('test_final_100k.csv')

In [120]:
merge = entrenamiento_df.merge(test_final_df,on=['idpostulante','idaviso'],how='inner')

In [121]:
merge.shape

(13893, 4)

In [122]:
merge.head()

,idaviso,idpostulante,postulacion,id
0,1112406254,5bQaeZ,0,33984
1,1112424848,NkeQYJ,0,43241
2,1112094756,VNr0xb2,0,14479
3,1112476948,86zN4M,0,92838
4,1112210505,eOv8mo,0,18189


In [123]:
merge['en_test_final']=1

In [124]:
merge = merge.drop(['postulacion','id'],1)

In [125]:
merge.head()

,idaviso,idpostulante,en_test_final
0,1112406254,5bQaeZ,1
1,1112424848,NkeQYJ,1
2,1112094756,VNr0xb2,1
3,1112476948,86zN4M,1
4,1112210505,eOv8mo,1


In [126]:
entrenamiento_df = entrenamiento_df.merge(merge,on=['idpostulante','idaviso'],how='outer')

In [127]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion,en_test_final
0,1112513555,NzrLwKJ,0,NaN
1,1111882745,N6qMwV,0,NaN
2,1112362954,Nzr3zvL,0,NaN
3,1112146217,jkj13j0,0,NaN
4,1112400202,ZDrmWxY,0,NaN


In [128]:
entrenamiento_df.en_test_final.hasnans

True

In [129]:
entrenamiento_df.en_test_final.fillna(0,inplace=True)

In [130]:
entrenamiento_df.shape

(6274687, 4)

In [131]:
entrenamiento_df = entrenamiento_df[entrenamiento_df.en_test_final==0]

In [132]:
entrenamiento_df.shape

(6260794, 4)

In [133]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion,en_test_final
0,1112513555,NzrLwKJ,0,0.0
1,1111882745,N6qMwV,0,0.0
2,1112362954,Nzr3zvL,0,0.0
3,1112146217,jkj13j0,0,0.0
4,1112400202,ZDrmWxY,0,0.0


In [135]:
entrenamiento_df.drop('en_test_final',1,inplace=True)

In [134]:
entrenamiento_df.postulacion.value_counts()

1    3263194
0    2997600
Name: postulacion, dtype: int64

In [136]:
#entrenamiento_df.to_csv('para_aplicar_features.csv')

In [70]:
entrenamiento_df=pd.read_csv('para_aplicar_features.csv')
entrenamiento_df.drop('Unnamed: 0',axis=1,inplace=True)

# Armo la Matriz de Features

In [68]:
del postulantes_edu_df
del postulantes_gye_df
del postulantes_edad_trucha
del avisos_detalle_missing
del postulaciones_df2
del merge

Tomamos un sample de tamaño limitado para optimizar el armado de Features.

In [71]:
entrenamiento_df = entrenamiento_df[entrenamiento_df.postulacion==1].sample(50000).append(entrenamiento_df[entrenamiento_df.postulacion==0].sample(50000))

In [72]:
entrenamiento_df.postulacion.value_counts()

1    50000
0    50000
Name: postulacion, dtype: int64

# (NO CORRER)Calculamos la edudacion promedio de los avisos en base a las postulaciones que tuvieron

Obtengo un dataframe con los avisos y sus postulaciones.

In [ ]:
merge_detalle_postulaciones_df = postulaciones_df.merge(avisos_detalle_filtrados_df, on = 'idaviso',how='inner')

In [ ]:
merge_detalle_postulaciones_df.head(2)

In [ ]:
postulantes = postulantes_filtrados_efectivos_df.groupby('idpostulante')

In [ ]:
dictionary = {}
def armarFeatureConValoracionEduPorAviso(idpostulante,idaviso):
    if not idaviso in dictionary:
        dictionary[idaviso] = [0,0]
    val_edu = dictionary[idaviso]
    val_edu[1]+=1
    val_edu[0] += postulantes.get_group(idpostulante)['valoracion_edu'].iloc[0]
    dictionary[idaviso] = val_edu
   

In [ ]:
merge_detalle_postulaciones_df.apply(lambda row: armarFeatureConValoracionEduPorAviso(row['idpostulante'], row['idaviso']),axis=1)

In [ ]:
for key,value in dictionary.items():
    prom = round((value[0]/value[1]))
    dictionary[key] = prom

In [ ]:
def assignValEduToAdd(c):
    return dictionary[c]

In [ ]:
merge_detalle_postulaciones_df['val_edu'] = merge_detalle_postulaciones_df['idaviso'].apply(assignValEduToAdd)

In [ ]:
merge_detalle_postulaciones_df.head()

# Al igual que hicimos con la educacion, buscamos la edad promedio de los postulantes a un aviso

In [ ]:
dictionary = {}
def armarFeatureConEdadPorAviso(idpostulante,idaviso):
    if not idaviso in dictionary:
        dictionary[idaviso] = [0,0] #El primer elemento es la suma acumulada de  edad. El segundo es contador
    val_edad = dictionary[idaviso]
    val_edad[1]+=1
    val_edad[0] += postulantes.get_group(idpostulante)['edad'].iloc[0]
    dictionary[idaviso] = val_edad
   

In [ ]:
merge_detalle_postulaciones_df.apply(lambda row: armarFeatureConEdadPorAviso(row['idpostulante'], row['idaviso']),axis=1)

In [ ]:
for key,value in dictionary.items():
    prom = round((value[0]/value[1]))
    dictionary[key] = prom

In [ ]:
def assignValEdadToAdd(c):
    return dictionary[c]

In [ ]:
merge_detalle_postulaciones_df['val_edad'] = merge_detalle_postulaciones_df['idaviso'].apply(assignValEdadToAdd)

In [ ]:
merge_detalle_postulaciones_df.head()

In [ ]:
merge_detalle_postulaciones_df = merge_detalle_postulaciones_df.merge(avisos_detalle_filtrados_df.drop(columns=['nombre_zona','tipo_de_trabajo','nivel_laboral','denominacion_empresa','nombre_area','titulo'],axis=1),how='inner',on='idaviso')

In [142]:
#merge_detalle_postulaciones_df.to_csv('merge_postulaciones_detalle_df.csv')

In [109]:
merge_detalle_postulaciones_df = pd.read_csv('merge_detalle_postulaciones_df.csv')

In [110]:
merge_detalle_postulaciones_df.drop('Unnamed: 0',1,inplace=True)

In [112]:
merge_detalle_postulaciones_df = merge_detalle_postulaciones_df.merge(avisos_detalle_filtrados_df.drop(['nombre_zona','tipo_de_trabajo','nivel_laboral','nombre_area','denominacion_empresa'],1),how='left',on='idaviso')

In [76]:
merge_detalle_postulaciones_df.head()

,idaviso,idpostulante,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1112248724,NjlD,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack
1,1112248724,MV6eM8N,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack
2,1112248724,5LzQkN,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack
3,1112248724,8M2rOBz,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack
4,1112248724,Dr6V3Da,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack


# Comenzamos a dar formato para armar los features

En primer lugar agrupo todos los avisos a los que se postulo un usuario

In [77]:
grupo_postulantes = merge_detalle_postulaciones_df.groupby('idpostulante')

En segundo lugar hago que 'idaviso' sea el indice para poder acceder de forma directa.

In [78]:
grupo_avisos = avisos_detalle_filtrados_df.groupby('idaviso')

Armo lista de los usuarios que se postularon a algun aviso del que yo tenga datos en avisos_detalle_filtrados_df

In [79]:
usuarios_con_postulaciones = grupo_postulantes.groups.keys()

Formato necesario para acceder de forma directa a los idpostulantes.

In [ ]:
postulantes = postulantes_filtrados_efectivos_df.groupby('idpostulante')

### Funcion para reducir el ruido en los features de area, empresa e interes titulo

In [80]:
def categorizarFeatures(num):
    if num>0 and num<=3:
        return 1
    if num>3 and num<=10:
        return 2
    if num>10:
        return 3
    return num

# Feature Postulacion A Area

## A ese Area En esa Zona

In [81]:
def armarFeaturePostulacionesUsuarioAEseAreaEnEsaZona(idpostulante,idaviso,ajuste):
    if (idpostulante not in usuarios_con_postulaciones):
        return 0
    area=grupo_avisos.get_group(idaviso)['nombre_area'].iloc[0]
    zona=grupo_avisos.get_group(idaviso)['nombre_zona'].iloc[0]
    postulaciones_de_usuario = grupo_postulantes.get_group(idpostulante).groupby('nombre_area')
    if area in postulaciones_de_usuario['nombre_area'].groups.keys():
        zonas=postulaciones_de_usuario.get_group(area)['nombre_zona'].value_counts()
        if zonas.index.contains(zona):
            return categorizarFeatures((zonas[zona]-ajuste))
    return 0

In [82]:
feature = entrenamiento_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEseAreaEnEsaZona(row['idpostulante'], row['idaviso'],row['postulacion']),axis=1)

In [83]:
entrenamiento_df['postulaciones_a_ese_area_en_esa_zona'] = pd.DataFrame(feature)

# Feature Postulacion A Empresa

In [84]:
def armarFeaturePostulacionesUsuarioAEsaEmpresa(idpostulante,idaviso,ajuste):
    if (idpostulante in usuarios_con_postulaciones) == False:
        return 0
    empresa = grupo_avisos.get_group(idaviso)['denominacion_empresa'].iloc[0]
    empresas = grupo_postulantes.get_group(idpostulante)['denominacion_empresa'].value_counts()
    if empresas.index.contains(empresa):
        return categorizarFeatures((empresas[empresa]-ajuste))
    return 0

In [85]:
feature = entrenamiento_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEsaEmpresa(row['idpostulante'], row['idaviso'],row['postulacion']),axis=1)

In [86]:
entrenamiento_df['postulaciones_a_esa_empresa'] = pd.DataFrame(feature)

# (NO CORRER)Feature Educacion

In [ ]:
g_avisos = merge_detalle_postulaciones_df.groupby('idaviso')

In [ ]:
def armarFeatureEduDelAvisoYPostulante(idpostulante,idaviso):
    if ((idpostulante in usuarios_con_postulaciones) == False) or  not (idaviso in g_avisos.groups.keys()) :
        return 0
    val_edu = g_avisos.get_group(idaviso)['val_edu'].iloc[0]
    valoracion_edu = postulantes.get_group(idpostulante)['valoracion_edu'].iloc[0]
    if abs(val_edu-valoracion_edu) < 3:
        return 1
    return 0

In [ ]:
feature = entrenamiento_df.apply(lambda row: armarFeatureEduDelAvisoYPostulante(row['idpostulante'], row['idaviso']),axis=1)

In [ ]:
entrenamiento_df['relacion_educacion_con_aviso'] = pd.DataFrame(feature)

# (NO CORRER)Feature Edad

In [ ]:
def armarFeatureEdadDelAvisoYPostulante(idpostulante,idaviso):
    if ((idpostulante in usuarios_con_postulaciones) == False) or  not (idaviso in g_avisos.groups.keys()) :
        return 0
    val_edad = g_avisos.get_group(idaviso)['val_edad'].iloc[0]
    edad = postulantes.get_group(idpostulante)['edad'].iloc[0]
    if abs(val_edad-edad) <=3:
        return 1
    return 0

In [ ]:
feature = entrenamiento_df.apply(lambda row: armarFeatureEduDelAvisoYPostulante(row['idpostulante'], row['idaviso']),axis=1)

In [ ]:
entrenamiento_df['relacion_edad_con_aviso'] = pd.DataFrame(feature)

# Feature Edad- Genero -Area

In [88]:
merge_detalle_postulaciones_y_usuarios=merge_detalle_postulaciones_df.merge(postulantes_filtrados_df,on='idpostulante')

In [92]:
merge_detalle_postulaciones_y_usuarios.drop(['nivel_laboral','tipo_de_trabajo','nombre_zona','denominacion_empresa','idpostulante','idaviso','titulo'],axis=1,inplace=True)

In [93]:
postulantes_filtrados_df=postulantes_filtrados_df.set_index('idpostulante')

In [94]:
grupo_por_edad_sexo = merge_detalle_postulaciones_y_usuarios.groupby(['edad','sexo'])

In [95]:
grupo_por_edad_sexo = grupo_por_edad_sexo.nombre_area.value_counts()

In [96]:
del merge_detalle_postulaciones_y_usuarios

In [97]:
cant_de_avisos_por_area = avisos_detalle_filtrados_df.nombre_area.value_counts()

In [98]:
grupo_por_edad_sexo.head()

edad  sexo  nombre_area                      
10    FEM   Ingeniería  Industrial               2
11    MASC  Tesorería                            2
12    MASC  Finanzas                             5
            Compras                              1
            Planeamiento económico-financiero    1
Name: nombre_area, dtype: int64

In [99]:
cant_de_avisos_por_area.head()

Ventas            3153
Comercial         1776
Administración    1706
Producción        1538
Programación       958
Name: nombre_area, dtype: int64

In [100]:
def obtener_valoracion_feature(area,sexo,edad):
    try:
        result = ((grupo_por_edad_sexo[edad][sexo][area]/cant_de_avisos_por_area[area])/((grupo_por_edad_sexo[edad][sexo]/cant_de_avisos_por_area).sum()))
    except (KeyError, IndexError):
        return 0
    return abs(result*1000)

In [148]:
def categorizarFeaturesEspeciales(num):
    if num<=0:
        return 0
    if num>0 and num<3:
        return 1
    if num>=3 and num<=7:
        return 2
    if num>7 and num<=20:
        return 3
    if num>20 and num <=35:
        return 4
    if num>35 and num <=50:
        return 5
    return 6

In [102]:
""" Funcion que obtiene en base a la edad, area y sexo, un puntaje especifico."""
def FeatureEdadGeneroArea(idpostulante,idaviso):
    area_aviso = grupo_avisos.get_group(idaviso)['nombre_area'].iloc[0]
    genero = postulantes_filtrados_df.loc[idpostulante]['sexo']
    edad = postulantes_filtrados_df.loc[idpostulante]['edad']
    resultado_parcial =  obtener_valoracion_feature(area_aviso,genero,edad)
    resultado = categorizarFeaturesEspeciales(resultado_parcial)
    return resultado



In [103]:
feature = entrenamiento_df.apply(lambda row: FeatureEdadGeneroArea(row['idpostulante'], row['idaviso']),axis=1)

In [104]:
entrenamiento_df['relacion_edad_genero_aviso'] = pd.DataFrame(feature.fillna(0))


# Feature Titulo

In [106]:
import unicodedata
def shave_marks(txt):
    """This method removes all diacritic marks from the given string"""
    norm_txt = unicodedata.normalize('NFD', txt)
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c))
    return unicodedata.normalize('NFC', shaved)

In [107]:
from collections import Counter
def armarFeatureInteresEnPalabrasDeTitulo(idpostulante, idaviso, ajuste):
    if (idpostulante in usuarios_con_postulaciones) == False:
        return 0
    palabras_en_titulos_de_sus_postulaciones = shave_marks(" ".join(grupo_postulantes.get_group(idpostulante)['titulo'].values.tolist()).lower())
    #Saco comas, parentesis, sufijos etc. Para abarcar mas palabras
    titulo_aviso = shave_marks(grupo_avisos.get_group(idaviso)['titulo'].iloc[0].lower()).replace("(",' ').replace(")",' ').replace(",",' ').replace('/',' ').replace(' para ',' ').replace('-',' ').replace(' con ',' ').replace('experiencia',' ').replace('zona',' ').replace('villa',' ').replace('barrio',' ').replace('ero ',' ').replace('era ',' ').replace('cion ',' ').replace('ativo ',' ').replace('ativa ',' ').replace('dor ',' ').replace('dora ',' ').replace('dores ',' ').replace('doras ',' ').replace('ista ',' ').replace('or ',' ').replace('ora ',' ').replace('ico ',' ').replace('ica ',' ').replace('icos ',' ').replace('icas ',' ').replace('time',' ').replace('area',' ').split(" ")
    contador=0
    for x in titulo_aviso:
        if  len(x)>2 and x!='' and palabras_en_titulos_de_sus_postulaciones.count(x) > 0:
            contador = contador + palabras_en_titulos_de_sus_postulaciones.count(x) - ajuste
    return categorizarFeatures(contador)

In [108]:
feature = entrenamiento_df.apply(lambda row: armarFeatureInteresEnPalabrasDeTitulo(row['idpostulante'], row['idaviso'],row['postulacion']),axis=1)

In [109]:
entrenamiento_df['interes_en_titulo_de_aviso'] = pd.DataFrame(feature)

# El usuario tuvo postulaciones cercanas al 15 de abril?

In [111]:
del merge_detalle_postulaciones_df

In [130]:
postulaciones_df['dias_antes_de_15_abr'] = (pd.to_datetime('2018-04-16 00:00:00') - pd.to_datetime(postulaciones_df.fechapostulacion))

In [133]:
postulaciones_df['dias_antes_de_15_abr']=postulaciones_df['dias_antes_de_15_abr'].dt.days

In [137]:
postulaciones_mas_cercanas=postulaciones_df.loc[postulaciones_df.groupby('idpostulante').dias_antes_de_15_abr.idxmin()]

In [138]:
postulaciones_mas_cercanas=postulaciones_mas_cercanas.groupby('idpostulante')

In [141]:
def featureCercaniaDeUltimaPostulacion(idpostulante):
    if idpostulante not in usuarios_con_postulaciones: return 100
    return postulaciones_mas_cercanas.get_group(idpostulante).dias_antes_de_15_abr.iloc[0]

In [142]:
feature = entrenamiento_df.apply(lambda row: featureCercaniaDeUltimaPostulacion(row['idpostulante']),axis=1)

In [149]:
entrenamiento_df['cercania_de_ultima_post']=pd.DataFrame(feature)
entrenamiento_df['cercania_de_ultima_post']=entrenamiento_df['cercania_de_ultima_post'].apply(categorizarFeaturesEspeciales)

In [150]:
#Cuanto mas lejos fue su ultima postulacion, la categoria es mas alta

In [151]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso,cercania_de_ultima_post
3364021,1112228704,59MlaZ,1,1,1,2,2,5
4723361,1112198634,Rz9JBG1,1,1,1,3,3,3
4320296,1112312606,LNK8VKP,1,1,1,3,3,3
5728871,1112290296,mzbz6jd,1,0,0,3,2,6
3534073,1112440482,6r5JXbO,1,0,0,3,1,3


### De manera de entrenar bien el algoritmo, a los usuarios cuya ultima postulacion fue hace mas de 35 dias los tomo como que no se van a postular

In [152]:
def corregirEntrenamiento(se_postulo,categoria_cercania):
    if categoria_cercania>=5:
        return 0
    return se_postulo

In [153]:
postulaciones_corregidas=entrenamiento_df.apply(lambda row: corregirEntrenamiento(row['postulacion'],row['cercania_de_ultima_post']),axis=1)

In [155]:
entrenamiento_df['postulacion']=pd.DataFrame(postulaciones_corregidas)

# Comenzamos a dar formato para realizar el Fit del algoritmo. Creamos X e y

In [166]:
#X = pd.read_csv('training_set.csv')
#X = X.drop(['Unnamed: 0','idpostulante','idaviso'],axis=1)

In [156]:
matrix_df = entrenamiento_df

In [157]:
matrix_df.head()

,idaviso,idpostulante,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso,cercania_de_ultima_post
3364021,1112228704,59MlaZ,0,1,1,2,2,5
4723361,1112198634,Rz9JBG1,1,1,1,3,3,3
4320296,1112312606,LNK8VKP,1,1,1,3,3,3
5728871,1112290296,mzbz6jd,0,0,0,3,2,6
3534073,1112440482,6r5JXbO,1,0,0,3,1,3


In [158]:
matrix_df.idaviso.shape

(100000,)

In [159]:
#Deberian ser iguales.
matrix_df.shape[0] == entrenamiento_df.shape[0]

True

In [160]:
#Ver que está ocupando memoria.
%who

Counter	 FeatureEdadGeneroArea	 agrupacionZona	 armarFeatureInteresEnPalabrasDeTitulo	 armarFeaturePostulacionesUsuarioAEsaEmpresa	 armarFeaturePostulacionesUsuarioAEseAreaEnEsaZona	 avisos_detalle_filtrados_df	 avisos_online_df	 avisos_online_df2	 
cant_de_avisos_por_area	 categorizarFeatures	 categorizarFeaturesEspeciales	 corregirEntrenamiento	 entrenamiento_df	 feature	 featureCercaniaDeUltimaPostulacion	 grupo_avisos	 grupo_por_edad_sexo	 
grupo_postulantes	 matrix_df	 np	 obtener_valoracion_feature	 pd	 postulaciones_corregidas	 postulaciones_df	 postulaciones_mas_cercanas	 postulantes_filtrados_df	 
random	 setCategory	 shave_marks	 unicodedata	 usuarios_con_postulaciones	 


Me quedo con el indice como "id" de la tupla postulante-aviso.

In [161]:
matrix_df.drop(columns=['idpostulante','idaviso'],axis=1,inplace=True)

In [162]:
matrix_df.head()

,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso,cercania_de_ultima_post
3364021,0,1,1,2,2,5
4723361,1,1,1,3,3,3
4320296,1,1,1,3,3,3
5728871,0,0,0,3,2,6
3534073,1,0,0,3,1,3


In [163]:
matrix_df['postulacion'].value_counts()

0    66492
1    33508
Name: postulacion, dtype: int64

In [164]:
X = matrix_df

In [165]:
X.head()

,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso,cercania_de_ultima_post
3364021,0,1,1,2,2,5
4723361,1,1,1,3,3,3
4320296,1,1,1,3,3,3
5728871,0,0,0,3,2,6
3534073,1,0,0,3,1,3


In [169]:
y = X['postulacion']
X.head()

,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso,cercania_de_ultima_post
3364021,0,1,1,2,2,5
4723361,1,1,1,3,3,3
4320296,1,1,1,3,3,3
5728871,0,0,0,3,2,6
3534073,1,0,0,3,1,3


In [170]:
X = X.drop('postulacion',axis=1)

In [171]:
X.shape

(100000, 5)

In [172]:
y.shape

(100000,)

# Test Final DF

In [173]:
test_final_df = pd.read_csv('test_final_100k.csv')

In [174]:
test_final_df.head(2)

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL


In [175]:
X.head()

,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso,cercania_de_ultima_post
3364021,1,1,2,2,5
4723361,1,1,3,3,3
4320296,1,1,3,3,3
5728871,0,0,3,2,6
3534073,0,0,3,1,3


In [255]:
#feature = test_final_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEsaZona(row['idpostulante'], row['idaviso'],0),axis=1)

In [166]:
#test_final_df['postulaciones_a_esa_zona'] = pd.DataFrame(feature)

In [258]:
feature = test_final_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEseAreaEnEsaZona(row['idpostulante'], row['idaviso'],0),axis=1)
test_final_df['postulaciones_a_ese_area_en_esa_zona']=pd.DataFrame(feature)

In [259]:
test_final_df.head(2)

,id,idaviso,idpostulante,postulaciones_a_ese_area_en_esa_zona
0,0,739260,6M9ZQR,0
1,1,739260,6v1xdL,2


In [260]:
feature = test_final_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEsaEmpresa(row['idpostulante'], row['idaviso'],0),axis=1)

In [261]:
test_final_df['postulaciones_a_esa_empresa']=pd.DataFrame(feature)

In [262]:
test_final_df.head(2)

,id,idaviso,idpostulante,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa
0,0,739260,6M9ZQR,0,0
1,1,739260,6v1xdL,2,0


In [ ]:
feature = test_final_df.apply(lambda row: armarFeatureEduDelAvisoYPostulante(row['idpostulante'], row['idaviso']),axis=1)

In [ ]:
test_final_df['relacion_educacion_con_aviso'] = pd.DataFrame(feature)

In [ ]:
test_final_df.head(2)

In [ ]:
feature = test_final_df.apply(lambda row: armarFeatureEdadDelAvisoYPostulante(row['idpostulante'], row['idaviso']),axis=1)

In [ ]:
test_final_df['relacion_edad_con_aviso'] = pd.DataFrame(feature)

In [266]:
feature = test_final_df.apply(lambda row: FeatureEdadGeneroArea(row['idpostulante'], row['idaviso']),axis=1)

In [267]:
test_final_df['relacion_edad_genero_aviso'] = pd.DataFrame(feature.fillna(0))

In [268]:
feature = test_final_df.apply(lambda row: armarFeatureInteresEnPalabrasDeTitulo(row['idpostulante'], row['idaviso'],0),axis=1)

In [269]:
test_final_df['interes_en_titulo_de_aviso']=pd.DataFrame(feature)

In [270]:
test_final_df.head(2)

,id,idaviso,idpostulante,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
0,0,739260,6M9ZQR,0,0,3,0
1,1,739260,6v1xdL,2,0,2,0


In [ ]:
feature = test_final_df.apply(lambda row: armarFeatureAtraccionGeneroArea(row['idpostulante'], row['idaviso']),axis=1)
test_final_df['atraccion genero-area']=pd.DataFrame((feature).fillna(0).astype(int))

In [ ]:
feature = entrenamiento_df.apply(lambda row: featureCercaniaDeUltimaPostulacion(row['idpostulante']),axis=1)
entrenamiento_df['cercania_de_ultima_post']=pd.DataFrame(feature)
entrenamiento_df['cercania_de_ultima_post']=entrenamiento_df['cercania_de_ultima_post'].apply(categorizarFeaturesEspeciales)

In [ ]:
test_final_df[(test_final_df['postulaciones_a_ese_area_en_esa_zona']==0) & (test_final_df['postulaciones_a_esa_empresa']==0) & (test_final_df['interes_en_titulo_de_aviso']==0)].shape

Chequeamos que esten todos los datos.


In [274]:
test_final_df.shape

(100000, 4)

In [271]:
test_final_df = test_final_df.drop(['id','idaviso','idpostulante'],axis=1)

In [272]:
test_final_df.head()

,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
0,0,0,3,0
1,2,0,2,0
2,0,0,3,0
3,0,0,1,0
4,0,0,1,0


In [273]:
#test_final_df.to_csv('test_final_df.csv')

In [384]:
test_final_df = pd.read_csv('test_final_df.csv')
test_final_df.drop('Unnamed: 0',1,inplace=True)

In [385]:
test_final_df.head()

,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
0,0,0,3,0
1,2,0,2,0
2,0,0,3,0
3,0,0,1,0
4,0,0,1,0


# Visualizacion del set de etrenamiento

In [419]:
import matplotlib.pyplot as plt


In [541]:
#plt.scatter(X[:,0], X[:,1], alpha=0.4, c=trainLabels)
#plt.title("Datos de 'entrenamiento'")
#plt.show()

In [31]:
X.head()

,postulaciones_a_esa_zona,postulaciones_a_ese_area,postulaciones_a_esa_empresa,relacion_educacion_con_aviso,relacion_edad_con_aviso,interes_en_titulo_de_aviso
341,205,124,1,1,0,182
905,17,23,0,1,0,53
2338,14,8,0,0,1,3
6305,3,0,1,0,0,2
16281,72,1,1,1,0,9


In [33]:
test_final_df.head()

,postulaciones_a_ese_area,postulaciones_a_esa_empresa,relacion_educacion_con_aviso,interes_en_titulo_de_aviso,postulaciones_a_esa_zona
0,0,0,0,0,0
1,5,0,4,0,4
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


# XGBoost

In [325]:
from sklearn.model_selection import cross_val_score

In [326]:
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [128]:
xgbclassiffier = xgb.XGBClassifier(max_depth=4, learning_rate=0.1)

In [39]:
scores = cross_val_score(xgbclassiffier,X,y)
scores.mean()  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8995355009472982

In [129]:
xgbclassiffier.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [132]:
pred = xgbclassiffier.predict_proba(test_final_df)

In [133]:
pred = pred[:,1]

In [157]:
list(zip(X[X.columns], xgbclassiffier.feature_importances_))

[('postulaciones_a_esa_zona', 0.20666666),
 ('postulaciones_a_ese_area', 0.15133333),
 ('postulaciones_a_esa_empresa', 0.12733333),
 ('relacion_educacion_con_aviso', 0.154),
 ('relacion_edad_con_aviso', 0.104666665),
 ('interes_en_titulo_de_aviso', 0.256)]

# Random Forest

In [332]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(criterion='entropy')

In [333]:
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [334]:
list(zip(X[X.columns], clf.feature_importances_))

[('postulaciones_a_ese_area_en_esa_zona', 0.3423077358479879),
 ('postulaciones_a_esa_empresa', 0.05697225981167721),
 ('relacion_edad_genero_aviso', 0.06916100238234987),
 ('interes_en_titulo_de_aviso', 0.5315590019579849)]

In [335]:
pred = clf.predict_proba(test_final_df)

pred = pred[:,1]

# Random Forest con Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
bc = BaggingClassifier(base_estimator=clf,n_estimators=10)

In [398]:
bc.fit(X,y)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=50, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [542]:
pred = bc.predict_proba(test_final_df)

In [ ]:
pred = pred[:,1]

# Perceptron

In [309]:
from sklearn.linear_model import Perceptron
ptron = Perceptron()

In [124]:
ptron.fit(X,y)

/home/luciano/.local/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [125]:
ptron.score(X,y)

0.89038

In [383]:
pred = ptron.predict(test_final_df)

In [384]:
#Si se quiere hacer un submit solo con perceptron, pasar directo al Armado de Submit

# Bagging con Perceptron

In [310]:
from sklearn.ensemble import BaggingClassifier
bc = BaggingClassifier(base_estimator=ptron,n_estimators=50)

In [ ]:
maximo = 0
indice=0
for index in range(1,100):
    bc = BaggingClassifier(base_estimator=ptron,n_estimators=index)
    bc.fit(X,y)
    if maximo < bc.score(X,y):
        maximo = bc.score(X,y)
        indice = index
print(indice,maximo)
    

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

...

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

In [311]:
bc.fit(X,y)

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

BaggingClassifier(base_estimator=Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=50, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [312]:
bc.score(X,y)

0.85385

In [313]:
pred = bc.predict_proba(test_final_df)

In [314]:
pred = pred[:,1]

# Perceptron multinivel

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier()

In [ ]:
mlp.fit(X,y) 

In [ ]:
pred = mlp.predict(test_final_df)

# KNN

In [327]:
from sklearn.neighbors import KNeighborsClassifier

In [328]:
knn = KNeighborsClassifier(n_neighbors=50)

In [329]:
knn.fit(X,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=50, p=2,
           weights='uniform')

In [330]:
knn.score(X,y)

0.523

In [331]:
knn.predict(test_final_df)

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

# Bagging KNN

In [ ]:
from sklearn.ensemble import BaggingClassifier
bcknn = BaggingClassifier(base_estimator=knn,n_estimators=20)

In [ ]:
bcknn.fit(X,y)

In [ ]:
bcknn.score(X,y)

In [ ]:
pred=bcknn.predict_proba(test_final_df)

# Naive_Bayes

In [364]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [365]:
matrix_df.head()

,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
4970039,1,0,0,4,0
3771362,1,1,0,6,2
3203967,1,1,1,4,2
6079459,1,0,0,4,0
3004755,1,1,0,2,3


In [366]:
X.head()

,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
4970039,0,0,4,0
3771362,1,0,6,2
3203967,1,1,4,2
6079459,0,0,4,0
3004755,1,0,2,3


In [367]:
X.shape

(100000, 4)

In [368]:
X.head()

,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
4970039,0,0,4,0
3771362,1,0,6,2
3203967,1,1,4,2
6079459,0,0,4,0
3004755,1,0,2,3


In [369]:
X.columns

Index(['postulaciones_a_ese_area_en_esa_zona', 'postulaciones_a_esa_empresa',
       'relacion_edad_genero_aviso', 'interes_en_titulo_de_aviso'],
      dtype='object')

In [ ]:
#from sklearn.preprocessing import StandardScaler

#features = ['nombre_zona', 'tipo_de_trabajo', 'nivel_laboral','nombre_area','sexo','nombre','estado','edad']
#X = X.loc[:, features].values
#X = StandardScaler().fit_transform(X)
#X

In [ ]:
#X = pd.DataFrame(X)
#X.columns = ['nombre_zona', 'tipo_de_trabajo', 'nivel_laboral','nombre_area','sexo','nombre','estado','edad']

#X.head()

In [370]:
y.value_counts()

1    50000
0    50000
Name: postulacion, dtype: int64

In [371]:
nb.fit(X, y)

GaussianNB(priors=None)

In [372]:
nb.score(X,y)

0.82867

In [373]:
X.shape

(100000, 4)

In [374]:
X.head()

,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
4970039,0,0,4,0
3771362,1,0,6,2
3203967,1,1,4,2
6079459,0,0,4,0
3004755,1,0,2,3


In [375]:
pred = nb.predict(test_final_df)

# Boosting

In [275]:
from sklearn.ensemble import AdaBoostClassifier



#.drop(['interes_en_titulo_de_aviso','relacion_educacion_con_aviso'],axis=1)

adaboost = AdaBoostClassifier(n_estimators=100)
 

In [276]:
from sklearn.model_selection import cross_val_score

In [277]:
X.head()

,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
4970039,0,0,4,0
3771362,1,0,6,2
3203967,1,1,4,2
6079459,0,0,4,0
3004755,1,0,2,3


In [278]:
scores = cross_val_score(adaboost,X,y)
scores.mean()  

0.8517600300820097

In [279]:
adaboost.fit(X,y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)

In [280]:
test_final_df.head()

,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
0,0,0,3,0
1,2,0,2,0
2,0,0,3,0
3,0,0,1,0
4,0,0,1,0


In [281]:
pred = adaboost.predict_proba(test_final_df)

In [282]:
pred = pred[:,1]

In [283]:
list(zip(X, adaboost.feature_importances_))

[('postulaciones_a_ese_area_en_esa_zona', 0.23),
 ('postulaciones_a_esa_empresa', 0.1),
 ('relacion_edad_genero_aviso', 0.41),
 ('interes_en_titulo_de_aviso', 0.26)]

# Gradient Boosting

In [346]:
from sklearn.ensemble import GradientBoostingClassifier

In [347]:
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=2, random_state=0)

In [348]:
gb.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=1.0, loss='deviance', max_depth=2,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [349]:
pred = gb.predict_proba(test_final_df)

In [350]:
pred = pred[:,1]

In [353]:
scores = cross_val_score(gb, X,y)
scores

array([0.85711286, 0.85546289, 0.85617425])

# ARMADO DE SUBMIT

In [376]:
test_final_df = pd.read_csv('test_final_100k.csv')

In [377]:
test_final_df['sepostulo'] = pd.DataFrame(pred)

In [378]:
submit = test_final_df.drop(['idaviso','idpostulante'],axis=1)

In [379]:
submit.set_index('id',inplace=True)

In [380]:
submit.head()

,sepostulo
id,
0,0
1,1
2,0
3,0
4,0


In [381]:
submit.sepostulo.value_counts()

0    55500
1    44500
Name: sepostulo, dtype: int64

In [382]:
submit.head()

,sepostulo
id,
0,0
1,1
2,0
3,0
4,0


In [383]:
submit.to_csv('Submit_file.csv')